# Hands On

## Prepare ENV

```SQL
CREATE EXTENSION pg_buffercache;
```

## PG_BUFFERCACHE Extension

### pg_buffercache View

- One row per buffer
- Unused are shown with NULL on all fields except bufferid
- Shared system catalog shown as belonging to DB zero
- Using 128MB (default) of shared_buffers with 8kB of block size, there are 16,384 buffers, so pg_buffercache has the same number of 16,384 rows.

```SQL
SELECT * FROM pg_buffercache;"
```

### Summary with pg_buffercache_summary

```SQL
SELECT pg_buffercache_summary();
```

### Usage count with pg_buffercache_usage_counts

```SQL
SELECT pg_buffercache_usage_counts();
```

## Cache Analysis Use Cases

```SQL
-- Get buffer count per relation
SELECT n.nspname, c.relname, count(*) AS buffers
FROM 
    pg_buffercache b JOIN pg_class c 
        ON b.relfilenode = pg_relation_filenode(c.oid) AND b.reldatabase IN (0, (SELECT oid FROM pg_database WHERE datname = current_database()))
    JOIN pg_namespace n ON n.oid = c.relnamespace
GROUP BY n.nspname, c.relname
ORDER BY 3 DESC
LIMIT 10;
```

```SQL
-- Get shared memory cache usage by database
WITH empty_handled AS (
    SELECT 
        CASE 
            WHEN c.reldatabase IS NULL THEN 'Unused'
            WHEN c.reldatabase = 0 THEN 'System Catalog'
            ELSE d.datname
        END AS database
    FROM 
        pg_buffercache AS c LEFT JOIN pg_database AS d ON c.reldatabase = d.oid
)

SELECT
    database,
    count(*) AS cached_blocks
FROM 
    empty_handled
GROUP BY database
ORDER BY database;

SELECT COUNT(*) AS total_chached_blocks FROM pg_buffercache;
```